In [ ]:
import numpy as np
from numpy import pi
from matplotlib import pyplot as plt
import time
%matplotlib inline

import pyqg
from pyqg import diagnostic_tools as tools

In [ ]:
L =  1000.e3     # length scale of box    [m]
Ld = 15.e3       # deformation scale      [m]
kd = 1./Ld       # deformation wavenumber [m^-1]
Nx = 64          # number of grid points

H1 = 500.        # layer 1 thickness  [m]
H2 = 1000.       # layer 2
H3 = 1750.       # layer 3

U1 = 0.05          # layer 1 zonal velocity [m/s]
U2 = 0.025         # layer 2
U3 = 0.00          # layer 3

rho1 = 1025.
rho2 = 1025.275
rho3 = 1025.640

amp = 100
xmode = 1
ymode = 1
X,Y = np.meshgrid(
            np.arange(0.5,Nx,1.)/Nx*L,
            np.arange(0.5,Nx,1.)/Nx*L )
hbot = amp* np.sin(2*np.pi*xmode*X/L) * np.sin(2*np.pi*ymode*Y/L)


rbg = 0
rek = 1.e-7       # linear bottom drag coeff.  [s^-1]
f0  = 0.0001236812857687059 # coriolis param [s^-1]
beta = 1.2130692965249345e-11 # planetary vorticity gradient [m^-1 s^-1]

Ti = Ld/(abs(U1))  # estimate of most unstable e-folding time scale [s]
dt = Ti/200.   # time-step [s]
tmax = 100*Ti      # simulation time [s]

In [ ]:
m = pyqg.LayeredModel(nx=Nx, nz=3, U=[U1,U2,U3], V=[0.,0.,0.], L=L, f=f0, beta=beta,
                         H=[H1,H2,H3], rho=[rho1,rho2,rho3], hbot=hbot, rek=rek, rbg=rbg,
                        dt=dt, tmax=tmax, twrite=5000, tavestart=Ti*10, ntd=1)

In [ ]:
sig = 1.e-7
qi = sig*np.vstack([np.random.randn(m.nx,m.ny)[np.newaxis,],
                    np.random.randn(m.nx,m.ny)[np.newaxis,],
                    np.random.randn(m.nx,m.ny)[np.newaxis,]])

In [ ]:
m.set_q(qi)

In [ ]:
t0 = time.time()
m.run()
t1 = time.time()
print("computation time is", t1-t0, "s")

In [ ]:
plt.figure()
plt.pcolormesh(m.x,m.y,m.hb[-1,:,:],cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('bathymetry [m]')

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(131)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[0,]+m.Qy[0]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 1 PV')

plt.subplot(132)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[1,]+m.Qy[1]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 2 PV')

plt.subplot(133)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,(m.q[2,]+m.Qy[2]*m.y)/(U1/Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Layer 3 PV')

In [ ]:
print("The first baroclinic deformation radius is", m.radii[1]/1.e3, "km")
print("The second baroclinic deformation radius is", m.radii[2]/1.e3, "km")

In [ ]:
pn = m.modal_projection(m.p)

In [ ]:
plt.figure(figsize=(18,4))

plt.subplot(131)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,pn[0]/(U1*Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('Barotropic streamfunction')

plt.subplot(132)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,pn[1]/(U1*Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('1st baroclinic streamfunction')

plt.subplot(133)
plt.pcolormesh(m.x/m.rd,m.y/m.rd,pn[2]/(U1*Ld),cmap='Spectral_r')
plt.xlabel(r'$x/L_d$')
plt.ylabel(r'$y/L_d$')
plt.colorbar()
plt.title('2nd baroclinic streamfunction')

In [ ]:
kr, kespec_1 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[0])
_, kespec_2 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[1])
_, kespec_3 = tools.calc_ispec(m,m.get_diagnostic('KEspec')[2])

plt.loglog( kr, kespec_1, '.-' )
plt.loglog( kr, kespec_2, '.-' )
plt.loglog( kr, kespec_3, '.-' )

plt.legend(['layer 1','layer 2', 'layer 3'], loc='lower left')
plt.ylim([1e-14,1e-6]); plt.xlim([m.kk.min(), m.kk.max()])
plt.xlabel(r'k (m$^{-1}$)'); plt.grid()
plt.title('Kinetic Energy Spectrum');

In [ ]:
kr, modal_kespec_1 = tools.calc_ispec(m,m.get_diagnostic('KEspec_modal')[0])
_,  modal_kespec_2 = tools.calc_ispec(m,m.get_diagnostic('KEspec_modal')[1])
_,  modal_kespec_3 = tools.calc_ispec(m,m.get_diagnostic('KEspec_modal')[2])

_,  modal_pespec_2 = tools.calc_ispec(m,m.get_diagnostic('PEspec_modal')[0])
_,  modal_pespec_3 = tools.calc_ispec(m,m.get_diagnostic('PEspec_modal')[1])

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.loglog( kr, modal_kespec_1, '.-' )
plt.loglog( kr, modal_kespec_2, '.-' )
plt.loglog( kr, modal_kespec_3, '.-' )

plt.legend(['barotropic ','1st baroclinic', '2nd baroclinic'], loc='lower left')
plt.ylim([1e-14,1e-6]); plt.xlim([m.kk.min(), m.kk.max()])
plt.xlabel(r'k (m$^{-1}$)'); plt.grid()
plt.title('Kinetic Energy Spectra');


plt.subplot(122)
plt.loglog( kr, modal_pespec_2, '.-' )
plt.loglog( kr, modal_pespec_3, '.-' )

plt.legend(['1st baroclinic', '2nd baroclinic'], loc='lower left')
plt.ylim([1e-14,1e-6]); plt.xlim([m.kk.min(), m.kk.max()])
plt.xlabel(r'k (m$^{-1}$)'); plt.grid()
plt.title('Potential Energy Spectra');

In [ ]:
_, APEgenspec =  tools.calc_ispec(m,m.get_diagnostic('APEgenspec'))
_, APEflux =  tools.calc_ispec(m,m.get_diagnostic('APEflux'))
_, KEflux =   tools.calc_ispec(m,m.get_diagnostic('KEflux'))
_, KEspec =   tools.calc_ispec(m,m.get_diagnostic('KEspec')[1]*m.M**2)

ebud = [ APEgenspec,
         APEflux,
         KEflux,
         -m.rek*(m.Hi[-1]/m.H)*KEspec ]
ebud.append(-np.vstack(ebud).sum(axis=0))
ebud_labels = ['APE gen','APE flux div.','KE flux div.','Diss.','Resid.']
[plt.semilogx(kr, term) for term in ebud]
plt.legend(ebud_labels, loc='upper right')
plt.xlim([m.kk.min(), m.kk.max()])
plt.xlabel(r'k (m$^{-1}$)'); plt.grid()
plt.title('Spectral Energy Transfers');

# workspace

In [ ]:
m.H

In [ ]:
m.f

In [ ]:
H=[H1,H2,H3]

In [ ]:
m.Hi

In [ ]:
m.hb[0,:,:].max()

In [ ]:
m.hb[1,:,:].max()

In [ ]:
m.hb[2,:,:].max()

In [ ]:
Ti/86400

In [ ]:
dt

In [ ]:
tmax